In [1]:
import pandas as pd
import numpy as np
from data_preprocessing_pipeline import *
from sklearn.pipeline import Pipeline, make_pipeline, FeatureUnion
pd.options.display.max_columns = None  # 显示所有列
pd.set_option('display.float_format', lambda x: '%.3f' % x)  # 取消科学计数法

NameError: name 'pkg_resources' is not defined

In [ ]:
traindata = pd.read_csv('./data/2_train.csv', encoding='utf-8', sep=';')
testdata = pd.read_csv('./data/2_test.csv', encoding='utf-8', sep=';')


traindata.drop('id', inplace=True, axis=1)
traindata['y'] = np.where(traindata['y'] == 'yes', 1, 0)

id =testdata['id']
x_test = testdata.drop(['id','y'],axis=1)
x_train = traindata.drop('y',axis=1)
y_train = traindata['y']

In [ ]:
x_train = x_train.replace(['?','.','-'],np.nan)
x_train[ 'age']=x_train[ 'age'].fillna(x_train['age'].median()).astype(float)
x_train['job']=x_train[ 'job'].fillna('not')
x_train['marital']=x_train[ 'marital'].fillna('not')
x_train[ 'education']=x_train[ 'education'].fillna('not')
x_train['default']=x_train[ 'default'].fillna('not')
x_train['housing']=x_train[ 'housing'].fillna('not')
x_train[ 'loan']=x_train[ 'loan'].fillna('not')
x_train[ 'education']=x_train[ 'education'].fillna('not')
x_train['contact']=x_train[ 'contact'].fillna('not')
x_train['month']=x_train[ 'month'].fillna('not')
x_train[ 'day_of_week']=x_train[ 'day_of_week'].fillna('not')
x_train[ 'duration']=x_train[ 'duration'].fillna(0).astype(int)
x_train[ 'campaign']=x_train[ 'campaign'].fillna(0).astype(int)
x_train[ 'pdays']=x_train[ 'pdays'].fillna(999).astype(int)
x_train[ 'previous']=x_train[ 'previous'].fillna(-1).astype(int)
x_train[ 'poutcome']=x_train[ 'poutcome'].fillna('not')
x_train[ 'emp.var.rate']=x_train[ 'emp.var.rate'].fillna('not')
x_train[ 'emp.var.rate']=x_train[ 'emp.var.rate'].fillna(x_train['emp.var.rate'].median()).astype(float)
x_train[ 'cons.price.idx']=x_train[ 'cons.price.idx'].fillna(x_train['cons.price.idx'].median()).astype(float)
x_train[ 'cons.conf.idx']=x_train[ 'cons.conf.idx'].fillna(x_train['cons.conf.idx'].median()).astype(float)
x_train[ 'euribor3m']=x_train[ 'euribor3m'].fillna(x_train['euribor3m'].median()).astype(float)
x_train[ 'nr.employed']=x_train[ 'nr.employed'].fillna(x_train['nr.employed'].median()) .astype(float)                                  

In [ ]:
x_test = x_test.replace(['?','.','-'],np.nan)
x_test[ 'age']=x_test[ 'age'].fillna(x_test['age'].median()).astype(float)
x_test['job']=x_test[ 'job'].fillna('not')
x_test['marital']=x_test[ 'marital'].fillna('not')
x_test[ 'education']=x_test[ 'education'].fillna('not')
x_test['default']=x_test[ 'default'].fillna('not')
x_test['housing']=x_test[ 'housing'].fillna('not')
x_test[ 'loan']=x_test[ 'loan'].fillna('not')
x_test[ 'education']=x_test[ 'education'].fillna('not')
x_test['contact']=x_test[ 'contact'].fillna('not')
x_test['month']=x_test[ 'month'].fillna('not')
x_test[ 'day_of_week']=x_test[ 'day_of_week'].fillna('not')
x_test[ 'duration']=x_test[ 'duration'].fillna(0).astype(int)
x_test[ 'campaign']=x_test[ 'campaign'].fillna(0).astype(int)
x_test[ 'pdays']=x_test[ 'pdays'].fillna(999).astype(int)
x_test[ 'previous']=x_test[ 'previous'].fillna(-1).astype(int)
x_test[ 'poutcome']=x_test[ 'poutcome'].fillna('not')
x_test[ 'emp.var.rate']=x_test[ 'emp.var.rate'].fillna('not')
x_test[ 'emp.var.rate']=x_test[ 'emp.var.rate'].fillna(x_test['emp.var.rate'].median()).astype(float)
x_test[ 'cons.price.idx']=x_test[ 'cons.price.idx'].fillna(x_test['cons.price.idx'].median()).astype(float)
x_test[ 'cons.conf.idx']=x_test[ 'cons.conf.idx'].fillna(x_test['cons.conf.idx'].median()).astype(float)
x_test[ 'euribor3m']=x_test[ 'euribor3m'].fillna(x_test['euribor3m'].median()).astype(float)
x_test[ 'nr.employed']=x_test[ 'nr.employed'].fillna(x_test['nr.employed'].median()) .astype(float)             

In [ ]:
pipe = Pipeline(
       [
           ('Dummy', one_hotdummies())
       ])

pipe.fit(x_train,y_train)
x_train =pipe.transform(x_train)

In [ ]:
x_test =pipe.transform(x_test)

In [ ]:
x_train =x_train.astype(float)

In [ ]:
from lgbm_model import *

lgb = lightgbm_model()
model=lgb.create_best_model(x_train,y_train)

In [ ]:
prob=lgb.get_best_pro(x_train,y_train,range_=1000)

In [ ]:
prob=lgb.get_best_pro(x_train,y_train,begin=4000,times=6000,range_=1)

In [ ]:
y_pre = lgb.get_predict(x_train,pro=prob)

In [ ]:
lgb.get_model_report(y_pre,y_train)

In [ ]:
y_test =lgb.get_predict(x_test,pro=prob)

In [ ]:
y_ = pd.concat([pd.DataFrame(id),pd.DataFrame(y_test)],axis=1)

In [ ]:
y_.to_csv("张钱东_results.csv",index=False)

# 下面是测试

In [ ]:
def get_kind(x: pd.Series, diff_limit: int = 8):
    x = x.astype('str')
    x = x.str.extract(r'(^(\-|)(?=.*\d)\d*(?:\.\d*)?$)')[0]
    x.dropna(inplace=True)
    if x.nunique() > diff_limit:
        kind = 'numeric'
    else:
        kind = 'categorical'
    return kind

In [ ]:
categorical_features_indices = []
# for col in x_train.columns:
#     kind = get_kind(x=x_train[col], diff_limit=10)
#     if kind == 'categorical':
#         categorical_features_indices.append(col)

In [ ]:
from sklearn.model_selection import train_test_split
X_train1, X_validation, y_train1, y_validation = train_test_split(x_train, y_train, train_size=0.7, random_state=1234)

In [ ]:
from catboost import CatBoostRegressor
model=CatBoostRegressor(iterations=50, depth=3, learning_rate=0.1, loss_function='RMSE')
model.fit(X_train1,y_train1,cat_features=categorical_features_indices,eval_set=(X_validation, y_validation),plot=True)

In [ ]:
def get_best_pro(model,X,y,begin=0,times=-1,range_=1):#当样本过大时，可使用参数进行定位
        if y.nunique()>2:
            return 0
        else:
            model_train = pd.DataFrame()
            model_train['correct'] = y  # y_train存在index 合并会出错
            model_train['yes_prob'] = model.predict(X)
            model_train.sort_values(by='yes_prob', ascending=False, inplace=True)  # ascending=False 倒叙
            model_train.reset_index(drop=True, inplace=True)
            pred = [0] * model_train.shape[0]
            f1_prob_dict = dict()
            if times < 0:
                times = model_train.shape[0] - 1
            for x in range(begin, times,range_):
                pred[:x] = [1]*x
                f1 = round(f1_score(y_pred=pred, y_true=model_train['correct'], average='binary', pos_label=1), 3)
                this_prob = model_train['yes_prob'][x:x + 2].mean()
                f1_prob_dict[this_prob] = f1
                print(str(x)+"/"+str(times)+" : "+ str(f1))
        
            f1_max = max(f1_prob_dict.values())
            prob_max = max(f1_prob_dict, key=f1_prob_dict.get)
            return prob_max

In [ ]:
pro = get_best_pro(model,x_train,y_train,begin=5000,times=8000,range_=1)

In [ ]:
model_proba = model.predict(x_train)  # 猜一类的机率
y_hat = [1 if y >= pro else 0 for y in model_proba]

In [ ]:
lgb.get_model_report(y_hat,y_train)

# 再做个测试，可能来不及了

In [ ]:
traindata = pd.read_csv('training_1.csv', encoding='utf-8')
testdata = pd.read_csv('test_1.csv', encoding='utf-8')


traindata.drop('id', inplace=True, axis=1)
traindata['y'] = np.where(traindata['y'] == 'yes', 1, 0)

id =testdata['id']
x_test = testdata.drop(['id','y'],axis=1)
x_train = traindata.drop('y',axis=1)
y_train = traindata['y']

In [ ]:
x_train = x_train.replace(['?','.','-'],np.nan)
x_train[ 'age']=x_train[ 'age'].fillna(x_train['age'].median()).astype(float)
x_train['job']=x_train[ 'job'].fillna('not')
x_train['marital']=x_train[ 'marital'].fillna('not')
x_train[ 'education']=x_train[ 'education'].fillna('not')
x_train['default']=x_train[ 'default'].fillna('not')
x_train['housing']=x_train[ 'housing'].fillna('not')
x_train[ 'loan']=x_train[ 'loan'].fillna('not')
x_train[ 'education']=x_train[ 'education'].fillna('not')
x_train['contact']=x_train[ 'contact'].fillna('not')
x_train['month']=x_train[ 'month'].fillna('not')
x_train[ 'day_of_week']=x_train[ 'day_of_week'].fillna('not')
x_train[ 'duration']=x_train[ 'duration'].fillna(0).astype(int)
x_train[ 'campaign']=x_train[ 'campaign'].fillna(0).astype(int)
x_train[ 'pdays']=x_train[ 'pdays'].fillna(999).astype(int)
x_train[ 'previous']=x_train[ 'previous'].fillna(-1).astype(int)
x_train[ 'poutcome']=x_train[ 'poutcome'].fillna('not')
x_train[ 'emp.var.rate']=x_train[ 'emp.var.rate'].fillna('not')
x_train[ 'emp.var.rate']=x_train[ 'emp.var.rate'].fillna(x_train['emp.var.rate'].median()).astype(float)
x_train[ 'cons.price.idx']=x_train[ 'cons.price.idx'].fillna(x_train['cons.price.idx'].median()).astype(float)
x_train[ 'cons.conf.idx']=x_train[ 'cons.conf.idx'].fillna(x_train['cons.conf.idx'].median()).astype(float)
x_train[ 'euribor3m']=x_train[ 'euribor3m'].fillna(x_train['euribor3m'].median()).astype(float)
x_train[ 'nr.employed']=x_train[ 'nr.employed'].fillna(x_train['nr.employed'].median()) .astype(float)     

In [ ]:
pipe = Pipeline(
       [
           ('Attribute_Selection', feature_select()),
           ('Dummy', one_hotdummies())
       ])

pipe.fit(x_train,y_train)
x_train =pipe.transform(x_train)
x_train =x_train.astype(float)

In [ ]:
from lgbm_model import *

lgb = lightgbm_model()
model=lgb.create_best_model(x_train,y_train)

In [ ]:
prob=lgb.get_best_pro(x_train,y_train,range_=1000)

In [ ]:
prob1=lgb.get_best_pro(x_train,y_train,begin=4000,times=6000,range_=1)

In [ ]:
y_pre1 = lgb.get_predict(x_train,pro=prob1)

In [ ]:
lgb.get_model_report(y_pre1,y_train)